# Pickhardt Payments Example

Example code demonstrating how to use the `pickhardtpayments` package in python. You need to install the library via `pip install pickhardtpayments` or you can download the full source code at https://ln.rene-pickhardt.de or a copy from github at: https://www.github.com/renepickhardt/pickhardtpayments

Of course you can use the classes in the library to create your own async payment loop or you could exchange the Oracle to talk to the actual Lightning network by wrapping against your favourite node implementation and exposing the the `send_onion` call. 

This example assumes a randomly generated Oracle to conduct payments in a simulated way. For this you will need an actual channelgraph which you can get for example with `lightning-cli listchannels > listchannels20220412.json`

## Acknowledgements & Funding
This work is funded via various sources including NTNU & BitMEX as well as many generous donors via https://donate.ln.rene-pickhardt.de or https://www.patreon.com/renepickhardt Feel free to go to my website at https://ln.rene-pickhardt.de to learn how I have been contributing to the open source community and why it is important to have independent open source contributors. In case you also wish to support me I will be very grateful

In [82]:
from pickhardtpayments.ChannelGraph import ChannelGraph
from pickhardtpayments.UncertaintyNetwork import UncertaintyNetwork
from pickhardtpayments.OracleLightningNetwork import OracleLightningNetwork
from pickhardtpayments.SyncSimulatedPaymentSession import SyncSimulatedPaymentSession

In [83]:
#Rene Pickhardt's public node key
RENE = "03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df"
#Carsten Otto's public node key
C_OTTO = "027ce055380348d7812d2ae7745701c9f93e70c1adeb2657f053f91df4f2843c71"

#we first need to import the chanenl graph from core lightning jsondump
#you can get your own data set via:
# $: lightning-cli listchannels > listchannels20220412.json
# alternatively you can go to https://ln.rene-pickhardt.de to find a data dump
channel_graph = ChannelGraph("../listchannels20220412.json")

#we now create an Oracle. This is a Simulated Network that assumes uniform distribution 
#of the liquidity for the channels on the `channel_graph`. Of course one could create one's own 
#oracle (for example a wrapper to an existing lightning network node / implementation)
oracle_lightning_network = OracleLightningNetwork(channel_graph)

In [84]:
# Since we randomly generated our Oracle but also since we control it we can compute
# The maximum possible amount that can be payed between two nodes
maximum_payable_amount =oracle_lightning_network.theoretical_maximum_payable_amount(RENE,C_OTTO,1000)
print(maximum_payable_amount, "sats would be possible on this oracle to deliver if including 1 sat basefee channels")


114108466.5 sats would be possible on this oracle to deliver if including 1 sat basefee channels


In [85]:
#Of course we want to restrict ourselves to the zeroBaseFee part of the network.
#Therefore we compute the theoretical maximum payable amount for that subgraph
maximum_payable_amount =oracle_lightning_network.theoretical_maximum_payable_amount(RENE,C_OTTO,0)
print(maximum_payable_amount, "sats possible on this oracle on the zeroBaseFeeGraph")

62732650.5 sats possible on this oracle on the zeroBaseFeeGraph


In [86]:
# We choose an amount that is half the theoretical maximum to demonstrate the
# minimum cost flow solver with Bayesian updates on the Uncertainty Network finds the
# liquidity rather quickly
tested_amount = int(maximum_payable_amount/2)
tested_amount = 27897763
print(tested_amount)
# From the channel graph we can derive our initial Uncertainty Network which is the main data structure
# that we maintain in order to deliver sats from one node to another
uncertainty_network = UncertaintyNetwork(channel_graph)

#we create a payment session which in this case operates by sending out the onions
#sequentially 
payment_session = SyncSimulatedPaymentSession(oracle_lightning_network, 
                                 uncertainty_network,
                                 prune_network=False)

#we need to make sure we forget all learnt information on the Uncertainty Nework
payment_session.forget_information()

#we run the simulation of pickhardt payments and track all the results
payment_session.pickhardt_pay(RENE,C_OTTO, tested_amount,mu=0,base=0)

27897763
09:19:56.246 | INFO | *** new pickhardt payment ***
09:19:57.754 | INFO | (01) 03ef-0311 Uncertainty Range:		[         0 ; 15,000,000]		inflight          0;	cond_cap: 15,000,000;	flow:  1,000,000
09:19:57.755 | INFO | (01) 03ef-02e9 Uncertainty Range:		[         0 ; 16,777,215]		inflight          0;	cond_cap: 16,777,215;	flow:    356,886
09:19:57.755 | INFO | (01) 03ef-02d4 Uncertainty Range:		[         0 ; 16,777,215]		inflight          0;	cond_cap: 16,777,215;	flow: 10,066,329
09:19:57.755 | INFO | (01) 03ef-0385 Uncertainty Range:		[         0 ;  5,000,000]		inflight          0;	cond_cap:  5,000,000;	flow:    796,094
09:19:57.756 | INFO | (01) 03ef-02f4 Uncertainty Range:		[         0 ; 10,811,137]		inflight          0;	cond_cap: 10,811,137;	flow:  3,355,443
09:19:57.756 | INFO | (01) 03ef-02e9 Uncertainty Range:		[         0 ; 16,777,215]		inflight          0;	cond_cap: 16,777,215;	flow:  3,000,000
09:19:57.756 | INFO | (01) 03ef-02e9 Uncertainty Range:		[         0 ; 16,7

## Optimizing for Fees

controlling mu we can decide how much we wish to focus on lower fees. However ,we will see that it will be much harder to deliver the same amount in the sense that we need to send out more onions and also have more failed attampts. Consequently we expect to need more time.

In [87]:
#we need to make sure we forget all learnt information on the Uncertainty Nework
payment_session.forget_information()

#we run the simulation of pickhardt payments and track all the results
payment_session.pickhardt_pay(RENE,C_OTTO, tested_amount,mu=100,base=0)

09:19:59.210 | INFO | *** new pickhardt payment ***
09:20:00.700 | INFO | (01) 03ef-0385 Uncertainty Range:		[         0 ;  5,000,000]		inflight          0;	cond_cap:  5,000,000;	flow:  2,000,000
09:20:00.700 | INFO | (01) 03ef-02e9 Uncertainty Range:		[         0 ; 16,777,215]		inflight          0;	cond_cap: 16,777,215;	flow:  1,000,000
09:20:00.700 | INFO | (01) 03ef-02f4 Uncertainty Range:		[         0 ; 10,811,137]		inflight          0;	cond_cap: 10,811,137;	flow:  4,906,248
09:20:00.701 | INFO | (01) 03ef-02f4 Uncertainty Range:		[         0 ; 10,811,137]		inflight          0;	cond_cap: 10,811,137;	flow:    740,000
09:20:00.701 | INFO | (01) 03ef-02e9 Uncertainty Range:		[         0 ; 16,777,215]		inflight          0;	cond_cap: 16,777,215;	flow:  2,000,000
09:20:00.701 | INFO | (01) 03ef-02f4 Uncertainty Range:		[         0 ; 10,811,137]		inflight          0;	cond_cap: 10,811,137;	flow:    998,574
09:20:00.702 | INFO | (01) 03ef-02f4 Uncertainty Range:		[         0 ; 10,811,137]		